In [1]:
from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# trajectories_df = pd.read_pickle('../AIS_preprocessed_data/trajectories_2019.pkl')

In [14]:
trajectories_df = load_filtered_data(years=('2019',), folder='../AIS_data', min_data_points=100)

def sort_row(row):
    # load into arrays
    e  = np.asarray(row['elapsed_s'])
    la = np.asarray(row['LAT'])
    lo = np.asarray(row['LON'])
    # get the sort‐order
    idx = np.argsort(e)
    # return a Series so apply(...) will build a DataFrame with matching columns
    return pd.Series({
        'elapsed_s': e[idx],
        'LAT'      : la[idx],
        'LON'      : lo[idx]
    })

# apply and overwrite in one go
trajectories_df[['elapsed_s','LAT','LON']] = trajectories_df.apply(sort_row, axis=1)

In [61]:
trajectories_df['Label'].value_counts()

Label
0    295090
1      1697
Name: count, dtype: int64

In [16]:
import numpy as np
import pandas as pd

# threshold in seconds
THRESHOLD = 3600

def split_row_into_segments(row, threshold=THRESHOLD):
    """
    Given a row with list‐columns ['elapsed_s','LAT','LON'],
    split into multiple segments whenever elapsed_s gaps > threshold.
    Returns a list of dicts, each dict is one sub-trajectory including break_duration.
    """
    
    e  = row['elapsed_s']
    la = row['LAT']
    lo = row['LON']

    # compute gaps and find break points
    gaps = np.diff(e)
    break_idxs = np.where(gaps > threshold)[0]

    # define segment boundaries: starts at 0, each break+1, ends at len(e)
    boundaries = np.concatenate([[0], break_idxs + 1, [len(e)]])

    segments = []
    e_arr = np.asarray(e)

    for i in range(len(boundaries) - 1):
        start, end = boundaries[i], boundaries[i + 1]
        seg = {
            'MMSI':        row['MMSI'],
            'VesselType':  row['VesselType'],
            'Label':       row['Label'],
            'elapsed_s':   e_arr[start:end] - e_arr[start],  # normalize to start at 0
            'LAT':         la[start:end],
            'LON':         lo[start:end],
            'weekday':     row['weekday'],
            'date':        row['date'],
        }

        # compute break_duration until next segment, or zero if last
        if i < len(boundaries) - 2:
            next_start = e_arr[boundaries[i+1]]
            current_end = e_arr[boundaries[i+1] - 1]
            seg['break_duration'] = int(next_start - current_end)
        else:
            seg['break_duration'] = 0

        # only keep segments with at least two points
        if len(seg['elapsed_s']) > 100:
            segments.append(seg)

    return segments

# apply to all rows and flatten
all_segments = []
for _, row in trajectories_df.iterrows():
    all_segments.extend(split_row_into_segments(row))

# build new DataFrame
segmented_df = pd.DataFrame(all_segments).reset_index(drop=True)


In [62]:
segmented_df['Label'].value_counts()

Label
0    300108
1      1598
Name: count, dtype: int64

In [18]:
trip_counts = segmented_df.groupby('MMSI').size().rename('num_trips')
segmented_df = segmented_df.merge(trip_counts, on='MMSI')

# define which labels are weekdays vs. weekend
weekday_labels = ['Monday','Tuesday','Wednesday','Thursday','Friday']
weekend_labels = ['Saturday','Sunday']

# weekday trip count per MMSI
segmented_df['weekday_trip_count'] = (
    segmented_df
      .groupby('MMSI')['weekday']
      .transform(lambda days: days.isin(weekday_labels).sum())
)

# weekend trip count per MMSI
segmented_df['weekend_trip_count'] = (
    segmented_df
      .groupby('MMSI')['weekday']
      .transform(lambda days: days.isin(weekend_labels).sum())
)



In [19]:
segmented_df

,MMSI,VesselType,Label,elapsed_s,LAT,LON,weekday,date,break_duration,num_trips,weekday_trip_count,weekend_trip_count
0,367659930,31.0,0,"[0, 69, 138, 210, 279, 349, 420, 488, 570, 639...","[30.4289, 30.42774, 30.42675, 30.42573, 30.424...","[-87.99302, -87.99258, -87.9919, -87.99129, -8...",Thursday,2019-01-10,0,166,119,47
1,367553360,30.0,0,"[0, 70, 149, 220, 301, 379, 449, 559, 630, 709...","[29.01648, 29.01666, 29.01664, 29.01661, 29.01...","[-91.83069, -91.83175, -91.83304, -91.83416, -...",Thursday,2019-01-10,0,250,176,74
2,367461560,90.0,0,"[0, 65, 131, 197, 263, 335, 402, 467, 533, 600...","[29.38964, 29.38584, 29.38206, 29.37743, 29.37...","[-91.36791, -91.37142, -91.37496, -91.37891, -...",Thursday,2019-01-10,0,270,191,79
3,538007067,70.0,0,"[0, 120, 301, 483, 662, 843, 1023, 1204, 1383,...","[28.82299, 28.82325, 28.8236, 28.82388, 28.824...","[-89.33299, -89.33357, -89.3343, -89.33491, -8...",Thursday,2019-01-10,0,4,3,1
4,369053000,90.0,0,"[0, 70, 139, 209, 270, 340, 410, 473, 540, 610...","[30.18058, 30.17847, 30.17641, 30.17425, 30.17...","[-88.56405, -88.56745, -88.57083, -88.57432, -...",Thursday,2019-01-10,0,41,29,12
...,...,...,...,...,...,...,...,...,...,...,...,...
301701,538006166,80.0,0,"[0, 81, 224, 301, 401, 510, 581, 651, 750, 831...","[28.91509, 28.91185, 28.90624, 28.90286, 28.89...","[-89.42226, -89.42518, -89.43059, -89.43154, -...",Sunday,2019-01-13,0,7,5,2
301702,538005865,80.0,0,"[0, 69, 130, 337, 440, 620, 681, 811, 891, 104...","[28.91911, 28.91597, 28.91333, 28.90537, 28.90...","[-89.4197, -89.422, -89.4239, -89.43129, -89.4...",Sunday,2019-01-13,0,6,5,1
301703,369293000,90.0,0,"[0, 100, 230, 309, 391, 480, 741, 811, 879, 90...","[29.07654, 29.07338, 29.06873, 29.06566, 29.06...","[-90.22863, -90.22913, -90.22987, -90.23059, -...",Sunday,2019-01-13,0,148,107,41
301704,636017526,70.0,0,"[0, 90, 161, 240, 307, 433, 498, 565, 636, 715...","[30.64235, 30.63707, 30.63251, 30.6274, 30.622...","[-88.03225, -88.03235, -88.03236, -88.03249, -...",Sunday,2019-01-13,0,28,22,6


In [20]:
weather_df = pd.read_csv('DailyWeatherData.csv')

In [23]:
import pandas as pd

# 1. Load
weather_df = pd.read_csv('DailyWeatherData.csv')

# 2. Drop any rows where WSPD, GST, WVHT or ATMP is NaN
weather_clean = weather_df.dropna(subset=['WSPD','GST','WVHT','ATMP'])

# 3. Group by date and compute the mean of those four columns
daily_avg = (
    weather_clean
      .groupby('date')[['WSPD','GST','WVHT','ATMP']]
      .mean()
      .reset_index()
)

print(daily_avg)


            date      WSPD       GST      WVHT       ATMP
0     2019-01-01  5.113460  6.320169  0.991576  18.358261
1     2019-01-02  6.913980  8.409737  1.157500  17.867721
2     2019-01-03  7.870577  9.824379  1.361417  16.082457
3     2019-01-04  7.620777  9.924995  1.101478  14.718592
4     2019-01-05  3.521667  4.828611  0.567667  15.243611
...          ...       ...       ...       ...        ...
1086  2021-12-27  5.953299  7.409201  1.084479  22.644965
1087  2021-12-28  6.732009  8.433191  1.286182  22.823268
1088  2021-12-29  5.687847  7.206597  1.484479  22.994097
1089  2021-12-30  3.135764  4.246354  1.347500  22.643750
1090  2021-12-31  4.519444  5.720486  1.105729  22.789757

[1091 rows x 5 columns]


In [26]:
import pandas as pd

# 1. Ensure your weather daily averages (daily_avg) are loaded:
# daily_avg = pd.read_csv(...) or from your previous code

# 2. Make sure both DataFrames share a common “date” column of the same type:
#    If segmented_df has a timestamp, extract the date part:
segmented_df['date'] = pd.to_datetime(segmented_df['date']).dt.date

#    And ensure daily_avg.DATE is also a date (not string):
daily_avg['date'] = pd.to_datetime(daily_avg['date']).dt.date

# 3. Merge on DATE (bringing in WSPD, GST, WVHT, ATMP averages):
merged_df = segmented_df.merge(
    daily_avg,
    on='date',
    how='left'    # or 'inner' if you only want segments with matching weather
)

# 4. (Optional) drop the extra DATE column if you don’t need it:
# merged_df = merged_df.drop(columns=['DATE'])

print(merged_df.head(5))


        MMSI  VesselType  Label  \
0  367659930        31.0      0   
1  367553360        30.0      0   
2  367461560        90.0      0   
3  538007067        70.0      0   
4  369053000        90.0      0   

                                           elapsed_s  \
0  [0, 69, 138, 210, 279, 349, 420, 488, 570, 639...   
1  [0, 70, 149, 220, 301, 379, 449, 559, 630, 709...   
2  [0, 65, 131, 197, 263, 335, 402, 467, 533, 600...   
3  [0, 120, 301, 483, 662, 843, 1023, 1204, 1383,...   
4  [0, 70, 139, 209, 270, 340, 410, 473, 540, 610...   

                                                 LAT  \
0  [30.4289, 30.42774, 30.42675, 30.42573, 30.424...   
1  [29.01648, 29.01666, 29.01664, 29.01661, 29.01...   
2  [29.38964, 29.38584, 29.38206, 29.37743, 29.37...   
3  [28.82299, 28.82325, 28.8236, 28.82388, 28.824...   
4  [30.18058, 30.17847, 30.17641, 30.17425, 30.17...   

                                                 LON   weekday        date  \
0  [-87.99302, -87.99258, -87.9919, -

In [63]:
merged_df.Label.value_counts()

Label
0    300108
1      1598
Name: count, dtype: int64

In [29]:
COASTAL_POINTS = [(25.9549238885927, -97.146341840721), 
                  (26.2105963742296, -97.1786084156481), 
                  (26.4392006473511, -97.2267948818937), 
                  (26.7131155219549, -97.3144205913896), 
                  (26.9409506241204, -97.3733392098903), 
                  (27.2381948299922, -97.3536997038123), 
                  (27.509565248758, -97.2499956220558), 
                  (27.8776471339216, -97.0077746348881), 
                  (28.0539999401663, -96.855022416767), 
                  (28.3280720859457, -96.4284079754034), 
                  (28.4969719023136, -96.1916422843358), 
                  (28.8635868411337, -95.3853300793036), 
                  (29.2108230117506, -94.9325270042829), 
                  (29.3231365306908, -94.7208564118929), 
                  (29.4049143402772, -94.7121277276709), 
                  (29.4827910105008, -94.5463471921147), 
                  (29.6592984918529, -94.0739071585848), 
                  (29.6792073576794, -93.9462501518341), (29.6630909605969, -93.826230743778), (29.7133276704338, -93.7574923555273), (29.7512255330155, -93.4399864669423), (29.7607479179518, -93.1998712835684), (29.6811534311097, -92.9380107569004), (29.5806227705967, -92.6565106907323), (29.5162520975399, -92.2642686354715), (29.5741538015925, -92.1475224839983), (29.5608678187247, -92.0318674180532), (29.4773159297318, -91.8529293914971), (29.4706667956712, -91.7951018585245), (29.5694090083582, -91.6969041610241), (29.6434023797613, -91.8496561349135), (29.7116557246348, -91.7492762663576), (29.7002833853108, -91.640167713579), (29.6196924687798, -91.6205281740788), (29.5181510398553, -91.5059641936618), (29.4754162216433, -91.4132219238004), (29.3746807827744, -91.3619409039944), (29.2243427925429, -91.2517412656884), (29.1044923894996, -90.7483800379835), (29.3044943130288, -90.5574400706218), (29.3301805074153, -90.4406939191486), (29.251198637516, -90.3915950703987), (29.2569102177412, -90.3206745110925), (29.1959702604612, -90.2977617150093), (29.0634922247878, -90.2584819182854), (29.1588178968435, -90.057722181173), (29.2997375962166, -89.8482337598385), (29.2768989322582, -89.6758422464489), (29.1788250621025, -89.4674449106425), (29.2064477008371, -89.3834316847671), (29.1559595208947, -89.3757940860725), (29.1511952028254, -89.4478057309062), (29.0062545714863, -89.4041623097948), (28.9489850882595, -89.4401681322116), (28.9021915070929, -89.3976157966283), (29.0224751597303, -89.2754142175166), (28.948030328351, -89.1401196120713), (29.034877420194, -89.1041137896545), (29.0406010378701, -89.0386486579873), (29.2083524286137, -88.954635072348), (29.4319111022806, -89.2448638227382), (29.4414136102532, -89.380157735065), (29.5316406824672, -89.4587158930652), (29.6872117675113, -89.2404987875087), (29.995091073275, -89.1464772356424), (30.1593768275681, -89.162843518559), (30.1744698581104, -89.2119423673089), (30.0706586329971, -89.3592389135599), (30.0914296066955, -89.4476168413101), (30.2791140829552, -89.3330528608931), (30.3803058562129, -88.9580522807037), (30.3671272452038, -88.8118468199808), (30.3304060353034, -88.7016471816741), (30.3680686334753, -88.6056316552292), (30.3360563490296, -88.585992115729), (30.3351146527177, -88.53143783934), (30.3125112244796, -88.4965231024513), (30.3247553955517, -88.4419688260617), (30.3887768822221, -88.3492265562003), (30.3482975768157, -88.1997478388939), (30.3125112244796, -88.1888369836164), (30.332289509439, -88.1408292203939), (30.2569223017108, -88.1190075098382), (30.2314728019458, -88.3437711285613), (30.2465547826037, -88.0786373453104), (30.2606920373758, -88.1091877400881), (30.337939714482, -88.1277361940602), (30.5436232982486, -88.0718067834058), (30.6450566630599, -88.0379831320445), (30.6506887357231, -87.9583338885163), (30.6121963694031, -87.9234191516275), (30.4157415341932, -87.921236980572), (30.3065333085294, -87.7750315198485), (30.2679047020761, -87.7652117500984), (30.2415154520466, -88.0368920465164), (30.2151191143229, -88.0259811912389), (30.238688023927, -87.6844707666455), (30.287692397843, -87.4542517202834), (30.297692397843, -87.4542517202834)]

In [32]:
# Convert to DataFrame with explicit column names
coast_df = pd.DataFrame(COASTAL_POINTS , columns=['LAT', 'LON'])

In [33]:
coast_df.to_csv('coastal_points.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c





# Extract NumPy arrays for vectorized operations
coast_latitudes = coast_df['LAT'].to_numpy()
coast_longitudes = coast_df['LON'].to_numpy()
num_coast_points = len(coast_df)

def compute_distance_to_coast(latitudes: np.ndarray, longitudes: np.ndarray) -> np.ndarray:
    """
    Compute the shortest distance (km) from each (latitude, longitude) pair 
    to the coastline polyline defined by COASTAL_POINTS.
    Returns an array of same shape as inputs.
    """
    # Ensure inputs are NumPy arrays and of matching shape
    # latitudes = np.asarray(latitudes)
    # longitudes = np.asarray(longitudes)
    # if latitudes.shape != longitudes.shape:
    #     raise ValueError("Latitude and longitude arrays must have the same shape.")

    # Compute distance from each point to every coastal vertex: shape (..., coast_point_count)
    all_distances = haversine_np(
        latitudes[..., None],       # extend dims for broadcasting
        longitudes[..., None],
        coast_latitudes[None, :],
        coast_longitudes[None, :]
    )

    # Find index and distance of the nearest coastal vertex (point A)
    nearest_vertex_index = np.argmin(all_distances, axis=-1)
    nearest_vertex_distance = np.take_along_axis(
        all_distances, 
        nearest_vertex_index[..., None], 
        axis=-1
    )[..., 0]

    # Identify adjacent vertices (A-1 and A+1), clipped to valid range
    prev_vertex_index = np.clip(nearest_vertex_index - 1, 0, num_coast_points - 1)
    next_vertex_index = np.clip(nearest_vertex_index + 1, 0, num_coast_points - 1)

    # Distances from query points to those adjacent vertices
    # Flatten indexing for arbitrary shapes
    flat_indices = np.arange(nearest_vertex_index.size).reshape(nearest_vertex_index.shape)
    prev_vertex_distance = all_distances[flat_indices, prev_vertex_index]
    next_vertex_distance = all_distances[flat_indices, next_vertex_index]

    # Choose the closer neighbor (point B)
    use_prev = prev_vertex_distance < next_vertex_distance
    neighbor_index = np.where(use_prev, prev_vertex_index, next_vertex_index)
    neighbor_distance = np.where(use_prev, prev_vertex_distance, next_vertex_distance)

    # Calculate distance between vertex A and B (segment length)
    lat_A = coast_latitudes[nearest_vertex_index]
    lon_A = coast_longitudes[nearest_vertex_index]
    lat_B = coast_latitudes[neighbor_index]
    lon_B = coast_longitudes[neighbor_index]
    segment_length = haversine_np(lat_A, lon_A, lat_B, lon_B)

    # Determine if the angle at A is obtuse (then closest is A itself)
    is_obtuse_at_A = neighbor_distance**2 > segment_length**2 + nearest_vertex_distance**2

    # Prepare output distances
    shortest_distances = np.empty_like(nearest_vertex_distance)

    # Direct distance for obtuse cases
    shortest_distances[is_obtuse_at_A] = nearest_vertex_distance[is_obtuse_at_A]

    # Perpendicular distance for acute cases via Heron's formula
    not_obtuse = ~is_obtuse_at_A
    if np.any(not_obtuse):
        da = nearest_vertex_distance[not_obtuse]
        db = neighbor_distance[not_obtuse]
        ab = segment_length[not_obtuse]
        s = 0.5 * (ab + da + db)  # semi-perimeter
        triangle_area = np.sqrt(s * (s - ab) * (s - da) * (s - db))
        shortest_distances[not_obtuse] = 2 * triangle_area / ab

    return shortest_distances


def extract_features(row):
    elapsed = np.array(row['elapsed_s']) / 60.0  # minutes
    lat = np.array(row['LAT'])
    lon = np.array(row['LON'])

    if len(elapsed) < 3:
        return pd.DataFrame([{}])  # skip short trips

    distances = haversine_np(lat[:-1], lon[:-1], lat[1:], lon[1:])
    distance_diffs = np.concatenate([[0], distances])
    dt = np.diff(elapsed) + 1e-10
    time_diffs = np.concatenate([[0], dt])
    cum_time = elapsed
    cum_dist = np.cumsum(distance_diffs)



    # Forward and backward speed calculations
    padded_time = np.concatenate([[cum_time[0] - 5], cum_time, [cum_time[-1] + 5]])
    padded_dist = np.concatenate([[cum_dist[0]], cum_dist, [cum_dist[-1]]])
    f_dist = interp1d(padded_time, padded_dist, fill_value='extrapolate')

    Time_step = 5
    forward_speed = (f_dist(cum_time + Time_step) - f_dist(cum_time)) / Time_step
    backward_speed = (f_dist(cum_time) - f_dist(cum_time - Time_step)) / Time_step

    

    

    pings_per_minute_in_movement = np.sum(forward_speed > 0.1) # G5 (the number of pings during the trip when the device is moving more than 0.1 km/min.  )
    min_speed = np.minimum(forward_speed, backward_speed)
    max_speed = np.maximum(forward_speed, backward_speed)

    stop_mask = min_speed < 0.025
    slow_mask = (min_speed >= 0.025) & (max_speed < 0.1)
    move_mask = max_speed >= 0.1

    total_time = cum_time[-1] - cum_time[0]
    break_duration = row['break_duration']        # G1 placeholder
    trip_duration = total_time  # G2 (Trip duration)
    movement_efficiency = np.sum(distance_diffs[forward_speed>0.1]) / cum_dist[-1] # G3 (the distance traveled when the device is moving more than 0.1 km/min. )

    trips_per_year = row['num_trips']  # G4 placeholder
    pings_in_movement = move_mask.sum()
    time_in_movement = time_diffs[move_mask].sum()
    pings_per_minute_in_movement = pings_in_movement / time_in_movement if time_in_movement > 0 else 0  # G5

    def get_durations(mask):
        durations, dur = [], 0
        for i, flag in enumerate(mask):
            dur = dur + time_diffs[i] if flag else 0
            if flag and (i == len(mask) - 1 or not mask[i + 1]):
                durations.append(dur)
        return durations

    stop_durations = get_durations(stop_mask)
    time_stopped = sum(stop_durations) # Total time stopped (min)
    pct_time_stopped = time_stopped / total_time if total_time > 0 else 0 # Percentage of time stopped
    number_of_stops = len(stop_durations) # Number of stops
    longest_stop = max(stop_durations) if stop_durations else 0 # Longest stop 
    shortest_stop = min(stop_durations) if stop_durations else 0 # Shortest stop

    slow_durations = get_durations(slow_mask)
    time_slow = sum(slow_durations) # Total time in slow movement (min) 
    pct_time_slow = time_slow / total_time if total_time > 0 else 0 # Percentage of time in slow movement (%) 
    number_of_slow = len(slow_durations) # Number of slow movements
    longest_slow = max(slow_durations) if slow_durations else 0 # Longest slow movement
    shortest_slow = min(slow_durations) if slow_durations else 0 # Shortest slow movement

    move_durations = get_durations(move_mask)
    time_move = sum(move_durations) # Total time in movement (min)
    pct_time_move = time_move / total_time if total_time > 0 else 0     # Percentage of time in movement (%)
    number_of_moves = len(move_durations)  # Number of movements
    longest_move = max(move_durations) if move_durations else 0 # Longest movement
    shortest_move = min(move_durations) if move_durations else 0 # Shortest movement
    avg_move_speed_kph = (distance_diffs[move_mask].sum() / time_diffs[move_mask].sum()) * 60 if np.any(move_mask) else 0
    max_move_speed_kph = max_speed[move_mask].max() * 60 if np.any(move_mask) else 0

    dist_move = distance_diffs[move_mask].sum()
    dist_slow = distance_diffs[slow_mask].sum()

    origin_lat, origin_lon = lat[0], lon[0]
    from_origin = haversine_np(origin_lat, origin_lon, lat, lon)
    dist_from_origin_in_stops = np.average(from_origin[stop_mask], weights=time_diffs[stop_mask]) if np.any(stop_mask) else 0
    total_distance = distance_diffs.sum()
    dist_from_origin = from_origin[-1]
    max_dist_from_origin = from_origin.max()


    distances = compute_distance_to_coast(lat, lon)  # This will compute the distance to coast for each point
    first_dist_coast = distances[0] # The first point distance from coast (km) 
    last_dist_coast = distances[-1] # The last point distance from coast (km) 
    max_dist_coast = np.max(distances) # Max distance traveled from coast (km)

    return pd.DataFrame([{
        'break_duration': break_duration,  # G1
        'trip_duration_min': trip_duration,  # G2
        'movement_efficiency': movement_efficiency,  # G3
        'trips_per_year': trips_per_year,  # G4
        'pings_per_minute_in_movement': pings_per_minute_in_movement,  # G5
        'time_stopped': time_stopped,
        'pct_time_stopped': pct_time_stopped,
        'number_of_stops': number_of_stops,
        'longest_stop': longest_stop,
        'shortest_stop': shortest_stop,
        'time_slow': time_slow,
        'pct_time_slow': pct_time_slow,
        'number_of_slow': number_of_slow,
        'longest_slow': longest_slow,
        'shortest_slow': shortest_slow,
        'time_move': time_move,
        'pct_time_move': pct_time_move,
        'number_of_moves': number_of_moves,
        'longest_move': longest_move,
        'shortest_move': shortest_move,
        'avg_move_speed_kph': avg_move_speed_kph,
        'max_move_speed_kph': max_move_speed_kph,
        'distance_move_km': dist_move,
        'distance_slow_km': dist_slow,
        'dist_from_origin_in_stops': dist_from_origin_in_stops,
        'total_distance_km': total_distance,
        'dist_from_origin_km': dist_from_origin,
        'max_dist_from_origin_km': max_dist_from_origin,
        'first_dist_coast_km': first_dist_coast,
        'last_dist_coast_km': last_dist_coast,
        'max_dist_coast_km': max_dist_coast,
        'weekday_trip_count': row['weekday_trip_count'],
        'weekend_trip_count': row['weekend_trip_count'],
        'gust_speed': row['GST'],
        'wave_height': row['WVHT'],
        'wind_speed': row['WSPD'],
        'air_temp': row['ATMP'],
        'Label': row['Label'],
    }])
data = pd.concat(merged_df.apply(extract_features, axis=1).tolist(), ignore_index=True)


/tmp/ipykernel_765637/3095571348.py:94: RuntimeWarning: invalid value encountered in divide
  shortest_distances[not_obtuse] = 2 * triangle_area / ab
/tmp/ipykernel_765637/3095571348.py:140: RuntimeWarning: invalid value encountered in scalar divide
  movement_efficiency = np.sum(distance_diffs[forward_speed>0.1]) / cum_dist[-1] # G3 (the distance traveled when the device is moving more than 0.1 km/min. )
/tmp/ipykernel_765637/3095571348.py:175: RuntimeWarning: invalid value encountered in scalar divide
  avg_move_speed_kph = (distance_diffs[move_mask].sum() / time_diffs[move_mask].sum()) * 60 if np.any(move_mask) else 0


In [74]:
data_clean.columns

Index(['break_duration', 'trip_duration_min', 'movement_efficiency',
       'trips_per_year', 'pings_per_minute_in_movement', 'time_stopped',
       'pct_time_stopped', 'number_of_stops', 'longest_stop', 'shortest_stop',
       'time_slow', 'pct_time_slow', 'number_of_slow', 'longest_slow',
       'shortest_slow', 'time_move', 'pct_time_move', 'number_of_moves',
       'longest_move', 'shortest_move', 'avg_move_speed_kph',
       'max_move_speed_kph', 'distance_move_km', 'distance_slow_km',
       'dist_from_origin_in_stops', 'total_distance_km', 'dist_from_origin_km',
       'max_dist_from_origin_km', 'first_dist_coast_km', 'last_dist_coast_km',
       'max_dist_coast_km', 'weekday_trip_count', 'gust_speed', 'wave_height',
       'wind_speed', 'air_temp', 'num_trips', 'Label'],
      dtype='object')

In [81]:
data_clean = data.dropna()

y = data_clean['Label']
X = data_clean.drop(columns=['Label',
                             'trips_per_year',
                             'num_trips',
                             'break_duration',
                            #  'weekday_trip_count',
                            #  'weekend_trip_count'
                             ])



In [82]:
np.sum(y)

np.int64(1597)

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_recall_curve,
    f1_score,
    classification_report
)


X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    stratify=y,
    test_size=0.20,
    random_state=42
)

# 2) Split train+val into train (60%) and val (20%)
# Since X_temp is 80%, using test_size=0.25 on it gives 0.25 * 0.8 = 0.20 overall
X_train_orig, X_val, y_train_orig, y_val = train_test_split(
    X_temp, y_temp,
    stratify=y_temp,
    test_size=0.25,
    random_state=42
)

# Define the base classifier.
# class_weight='balanced' automatically adjusts weights inversely proportional to class frequencies.
base_clf = RandomForestClassifier(
    class_weight='balanced',
    random_state=42,
    n_estimators=100
)

In [84]:
# --- Function to Train, Tune Threshold, and Evaluate ---
def train_tune_evaluate(X_train, y_train, X_val, y_val, X_test, y_test, classifier, sampling_strategy_name):
    """
    Trains a classifier, tunes its probability threshold on a validation set,
    and reports its performance on a test set.

    Args:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training labels.
        X_val (pd.DataFrame): Validation features (for threshold tuning).
        y_val (pd.Series): Validation labels (for threshold tuning).
        X_test (pd.DataFrame): Test features (for final evaluation).
        y_test (pd.Series): Test labels (for final evaluation).
        classifier (sklearn.base.Estimator): The machine learning classifier to use.
        sampling_strategy_name (str): A descriptive name for the current sampling strategy
                                      (e.g., "Original Distribution", "Undersampled", "Oversampled").
    """
    print(f"\n{'='*50}\n--- Results for {sampling_strategy_name} Training Data ---\n{'='*50}")

    clf = classifier.__class__(**classifier.get_params()) # Create a fresh instance to avoid retraining issues
    
    # Train the classifier on the provided training data
    print(f"Training classifier on {len(X_train)} samples...")
    clf.fit(X_train, y_train)
    print("Training complete.")

    # Get probability scores for the validation set
    val_scores = clf.predict_proba(X_val)[:, 1]

    # Compute Precision-Recall curve to find the best threshold
    precision, recall, thresholds = precision_recall_curve(y_val, val_scores)

    # Calculate F1-score for each threshold to identify the optimal one
    # Add a small epsilon (1e-6) to prevent division by zero for F1 calculation
    f1_scores = 2 * (precision[:-1] * recall[:-1]) / (precision[:-1] + recall[:-1] + 1e-6)

    # Find the threshold that yields the highest F1-score on the validation set
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    best_f1_val = f1_scores[best_idx]

    print(f"\nBest validation threshold = {best_threshold:.4f} (F1-score = {best_f1_val:.4f})")

    # Evaluate the classifier on the unseen test set using the best threshold found
    test_scores = clf.predict_proba(X_test)[:, 1]
    y_test_pred = (test_scores >= best_threshold).astype(int)

    print(f"\n--- Classification Report on Test Set @ Threshold {best_threshold:.4f} ---")
    print(classification_report(y_test, y_test_pred))

    return clf


In [85]:
import os
from joblib import dump
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


# make a directory for your models
os.makedirs("saved_models", exist_ok=True)

# Scenario 1: Original Distribution
model_orig = train_tune_evaluate(
    X_train_orig, y_train_orig,
    X_val, y_val,
    X_test, y_test,
    base_clf,
    "Original Distribution"
)
dump(model_orig, "saved_models/model_original_distribution.joblib")
print("Saved Original Distribution model to saved_models/model_original_distribution.joblib")

# Scenario 2: Undersampled Training Data
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train_orig, y_train_orig)

model_under = train_tune_evaluate(
    X_train_under, y_train_under,
    X_val, y_val,
    X_test, y_test,
    base_clf,
    "Undersampled"
)
dump(model_under, "saved_models/model_undersampled.joblib")
print("Saved Undersampled model to saved_models/model_undersampled.joblib")

# Scenario 3: Oversampled Training Data
oversampler = RandomOverSampler(random_state=42)
X_train_over, y_train_over = oversampler.fit_resample(X_train_orig, y_train_orig)

model_over = train_tune_evaluate(
    X_train_over, y_train_over,
    X_val, y_val,
    X_test, y_test,
    base_clf,
    "Oversampled"
)
dump(model_over, "saved_models/model_oversampled.joblib")
print("Saved Oversampled model to saved_models/model_oversampled.joblib")



--- Results for Original Distribution Training Data ---
Training classifier on 178132 samples...
Training complete.

Best validation threshold = 0.2100 (F1-score = 0.7303)

--- Classification Report on Test Set @ Threshold 0.2100 ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59059
           1       0.79      0.73      0.76       319

    accuracy                           1.00     59378
   macro avg       0.90      0.86      0.88     59378
weighted avg       1.00      1.00      1.00     59378

Saved Original Distribution model to saved_models/model_original_distribution.joblib

--- Results for Undersampled Training Data ---
Training classifier on 1916 samples...
Training complete.

Best validation threshold = 0.8500 (F1-score = 0.5522)

--- Classification Report on Test Set @ Threshold 0.8500 ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59059
           1       0